In [33]:
!tar cvfz container_hpo.tar.gz *

ann-churn.h5
Bayesian-OPT-Copy1.ipynb
Bayesian-OPT.ipynb
Build Container.ipynb
container/
container/scripts/
container/scripts/predictor.py
container/scripts/train
container/scripts/wsgi.py
container/scripts/.ipynb_checkpoints/
container/scripts/nginx.conf
container/scripts/serve
container/requirements.txt
container/local_test/
container/local_test/serve_local.sh
container/local_test/test_dir/
container/local_test/test_dir/input/
container/local_test/test_dir/input/config/
container/local_test/test_dir/input/config/resourceConfig.json
container/local_test/test_dir/input/config/hyperparameters.json
container/local_test/test_dir/output/
container/local_test/test_dir/output/success
container/local_test/test_dir/model/
container/local_test/test_dir/model/decision-tree-model.pkl
container/local_test/test_dir/model/ann-churn.pkl
container/local_test/train_local.sh
container/local_test/predict.sh
container/Dockerfile
container/.ipynb_checkpoints/
container/build_and_push.sh
container_hpo.tar.

In [14]:
def loadmodel(weightFile):
    ###############
    reg = Sequential()
    reg.add(Dense(1000, kernel_initializer='normal',input_dim = 28, activation='relu'))
    reg.add(Dense(512, kernel_initializer='normal',activation='relu'))
    reg.add(Dense(256, kernel_initializer='normal',activation='relu'))
    reg.add(Dense(128, kernel_initializer='normal',activation='relu'))
    reg.add(Dense(64, kernel_initializer='normal',activation='relu'))
    reg.add(Dense(1, kernel_initializer='normal',activation='tanh'))
    reg.load_weights(weightFile)
    return reg

In [ ]:
model = loadmodel(os.path.join(model_path, 'ann-churn.h5'))

In [24]:
# later...
from keras.models import model_from_json

# load json and create model
json_file = open('saved/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("saved/model.h5")
print("Loaded model from disk")


Loaded model from disk


In [26]:
from pickle import load

scaler = load(open('scaler.pkl', 'rb'))

train = pd.read_csv('SL_test_DG_mod.csv')

train = train.dropna()

train_x = train.drop(['PE_'], axis=1)
train_y = train['PE_']

train_x = train_x.astype('float32')
train_y = train_y.astype('float32')

#######
train_x = scaler.transform(train_x)

# Feature Scaling
# sc = StandardScaler()
pred = model.predict(train_x)

In [27]:
pred

array([[0.47818068],
       [0.4640664 ],
       [0.5198002 ],
       ...,
       [0.5362443 ],
       [0.42633906],
       [0.55470717]], dtype=float32)

In [29]:
endpointName = 'garrett-2020-06-16-15-27-13-763'
import boto3
from pickle import load

# Process and prepare the data
def transform_data(dataset):
    #######
    train = dataset.dropna()

    train_x = train.drop(['PE_'], axis=1)
    train_y = train['PE_']

    train_x = train_x.astype('float32')
    train_y = train_y.astype('float32')
    
    #######
    
    # Feature Scaling
    

    scaler = load(open('scaler.pkl', 'rb'))
    train_x = scaler.fit_transform(train_x)
    return pd.DataFrame(train_x)

test_X = pd.read_csv('SL_test_DG_mod.csv')

test_X = transform_data(test_X)


test_file = io.StringIO()
test_X.to_csv(test_file, header=None, index=None)

# Talk to SageMaker
client = boto3.client('sagemaker-runtime')
response = client.invoke_endpoint(
    EndpointName=endpointName,
    Body=test_file.getvalue(),
    ContentType='text/csv',
    Accept='Accept'
)

print(response['Body'].read().decode('ascii'))

0.74546367
0.7832163
0.773398
0.87126964
0.8433048
0.8488627
0.84317255
0.81183517
0.804632
0.7314138
0.79662305
0.7807538
0.6888113
0.679356
0.8098344
0.7976103
0.8646001
0.67924666
0.71988076
0.8510716
0.7865933
0.7643474
0.74304545
0.732974
0.71434945
0.6484137
0.77961344
0.7976927
0.8206988
0.81656414
0.7229775
0.7918747
0.6855863
0.6965654
0.8161279
0.8052634
0.811487
0.68158895
0.75178146
0.7320727
0.7035331
0.8107074
0.74004173
0.6581088
0.7216762
0.80022305
0.77562714
0.7293421
0.88311696
0.8003085
0.80599195
0.7367843
0.7497773
0.73196757
0.6404206
0.7079147
0.65720105
0.7950527
0.7462762
0.87406045
0.6961318
0.6849916
0.70373476
0.71986955
0.6402141
0.69754547
0.8653342
0.7973728
0.5992421
0.798115
0.79214084
0.8081021
0.8075412
0.6822216
0.85887
0.84589624
0.60541606
0.8296208
0.77457386
0.7929527
0.7683294
0.7205027
0.8635656
0.85882807
0.7155224
0.8315885
0.7926973
0.770294
0.80160856
0.77385044
0.6977182
0.6590751
0.76811546
0.5783476
0.77261287
0.76690114
0.78508866
0.77

In [30]:
test_X?

In [34]:
a?

Object `a` not found.


# Lambda

In [61]:
import json
import boto3
import os
import logging
import urllib
import io
import numpy as np
# import pickle
import pandas as pd
from contextlib import closing
from pickle import load

# scaler = load(open('scaler.pkl', 'rb'))

logger = logging.getLogger()
logger.setLevel(logging.INFO)

AREA_RATIO = [0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0]
SPEED = [93700, 113900, 134100, 154300, 174500, 194700, 214900]

def lambda_handler(event):    
    print('successfull call lambda_handler now',event)

    params = eval(event["params"])
    logger.info('params: {}'.format(params))
    params = [float(i) for i in params]
    # endpoint_name = 'sagemaker-tensorflow-2020-05-29-16-08-40-101'
    endpointName = 'garrett-2020-06-23-00-24-43-607'

    client = boto3.client('runtime.sagemaker')
    ############################
    k =1
    all_data = []
    xaxis = []

    for i in AREA_RATIO:
        for j in SPEED:
            a = params + [i,j]
            all_data.append(a)
            xaxis.append([str(i)+', '+str(j)])
    # all_data = scaler.transform(all_data)
    # all_data = all_data.tolist()

    logger.info('xaxis: {}'.format(xaxis))
    logger.info('all_data[0]: {}'.format(all_data[0]))

    all_data = pd.DataFrame(np.array(all_data))
    test_file = io.StringIO()
    all_data.to_csv(test_file, header=None, index=None)

    # Talk to SageMaker
    client = boto3.client('sagemaker-runtime')
    response = client.invoke_endpoint(
        EndpointName=endpointName,
        Body=test_file.getvalue(),
        ContentType='text/csv',
        Accept='Accept'
    )

    pred = response['Body'].read().decode('ascii')
    print(pred)

    # pred = client.invoke_endpoint(EndpointName=endpoint_name, Body=json.dumps(all_data))
    # pred = json.loads(pred['Body'].read().decode("utf-8"))
    # pred = pred['outputs']['score']['floatVal']
    
    return {
        'statusCode': 200,
        'xaxis' : json.dumps(xaxis),
        'AREA_RATIO' : json.dumps(AREA_RATIO),
        'SPEED' : json.dumps(SPEED),
        'preds': json.dumps(pred)}



In [62]:
event = {
  "params": "[100.5, 117.4000015258789, 7.300000190734863, 2.9000000953674316, 3.299999952316284, 2.4000000953674316, 1.2999999523162842, 1.7999999523162842, 50.0, 0.14499999582767487, 0.23399999737739563, 0.3659999966621399, 1.0299999713897705, 3.0999999046325684, 17.5, 7.0, -18.5, -27.5, -28.5, 0.10000000149011612, -29.5, -44.0, -52.0, 10.5, 0.6000000238418579, -73.0]"
}

In [63]:
len(eval(event['params']))

26

In [64]:
lambda_handler(event)

successfull call lambda_handler now {'params': '[100.5, 117.4000015258789, 7.300000190734863, 2.9000000953674316, 3.299999952316284, 2.4000000953674316, 1.2999999523162842, 1.7999999523162842, 50.0, 0.14499999582767487, 0.23399999737739563, 0.3659999966621399, 1.0299999713897705, 3.0999999046325684, 17.5, 7.0, -18.5, -27.5, -28.5, 0.10000000149011612, -29.5, -44.0, -52.0, 10.5, 0.6000000238418579, -73.0]'}
0.7513404
0.73758537
0.72576225
0.7100778
0.69597936
0.6786843
0.6518586
0.7797066
0.76902044
0.75591147
0.74313563
0.7331241
0.7195874
0.6970369
0.79954535
0.7901364
0.7816141
0.7712438
0.7619729
0.7454516
0.7194436
0.81880075
0.8124648
0.8038909
0.7960073
0.78444916
0.7605214
0.71954405
0.8365419
0.8301322
0.8231977
0.8144091
0.7993242
0.76195836
0.7028263
0.846545
0.8386305
0.82863885
0.8175032
0.791481
0.7276696
0.65867585
0.84084535
0.83346456
0.8249475
0.8091433
0.7551916
0.68137133
0.6046959
0.8315685
0.82342297
0.8089565
0.77260387
0.70146453
0.6269061
0.5565283
0.8122493
0.7

{'statusCode': 200,
 'xaxis': '[["0.2, 93700"], ["0.2, 113900"], ["0.2, 134100"], ["0.2, 154300"], ["0.2, 174500"], ["0.2, 194700"], ["0.2, 214900"], ["0.25, 93700"], ["0.25, 113900"], ["0.25, 134100"], ["0.25, 154300"], ["0.25, 174500"], ["0.25, 194700"], ["0.25, 214900"], ["0.3, 93700"], ["0.3, 113900"], ["0.3, 134100"], ["0.3, 154300"], ["0.3, 174500"], ["0.3, 194700"], ["0.3, 214900"], ["0.35, 93700"], ["0.35, 113900"], ["0.35, 134100"], ["0.35, 154300"], ["0.35, 174500"], ["0.35, 194700"], ["0.35, 214900"], ["0.4, 93700"], ["0.4, 113900"], ["0.4, 134100"], ["0.4, 154300"], ["0.4, 174500"], ["0.4, 194700"], ["0.4, 214900"], ["0.5, 93700"], ["0.5, 113900"], ["0.5, 134100"], ["0.5, 154300"], ["0.5, 174500"], ["0.5, 194700"], ["0.5, 214900"], ["0.6, 93700"], ["0.6, 113900"], ["0.6, 134100"], ["0.6, 154300"], ["0.6, 174500"], ["0.6, 194700"], ["0.6, 214900"], ["0.7, 93700"], ["0.7, 113900"], ["0.7, 134100"], ["0.7, 154300"], ["0.7, 174500"], ["0.7, 194700"], ["0.7, 214900"], ["0.8, 937